In [1]:
!pip install ultralytics
!pip3 install opencv-python
!python3 -m pip install --upgrade Pillow
!pip install mega.py

import numpy as np
import cv2
import os



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0


In [10]:
from mega import Mega
import zipfile

In [2]:
mega = Mega()
m = mega.login()
print("Downloading ...")
try:
    m.download_url('https://mega.nz/file/1Dg0iaAA#0ZRzdon9F7j0jiCjWHfUh-JNoxw5zfcmzceKrh_VySo')
except PermissionError:
    pass
with zipfile.ZipFile("171.zip","r") as zip_ref:
    zip_ref.extractall("scss-net-data")
try:
    m.download_url('https://mega.nz/file/sOBGiTwR#fjvzvxoLDFDKLv8EtLPBGqPUCo_T_I7y8f14n4eGu94')
except PermissionError:
    pass
with zipfile.ZipFile("193.zip","r") as zip_ref:
    zip_ref.extractall("scss-net-data")
print("DONE")

DONE


In [3]:
# move ch data
!mkdir -p data/ch-train\(zooniverse\)
!mkdir -p data/ch-train\(region_growth\)
!mkdir -p data/ch-test\(spoca\)
!mkdir -p data/ch-test\(region_growth\)

!cp -r scss-net-data/193_train/193\(zooniverse\)/imgs data/ch-train\(zooniverse\)/images 
!cp -r scss-net-data/193_train/193\(zooniverse\)/masks data/ch-train\(zooniverse\)/masks

!cp -r scss-net-data/193_train/193\(region_growth\)/imgs data/ch-train\(region_growth\)/images
!cp -r scss-net-data/193_train/193\(region_growth\)/masks data/ch-train\(region_growth\)/masks

!cp -r scss-net-data/193_test/masks\(spoca\) data/ch-test\(spoca\)/masks
!cp -r scss-net-data/193_test/masks\(region\) data/ch-test\(region_growth\)/masks

!cp -r scss-net-data/193_test/imgs data/ch-test\(spoca\)/images
!cp -r scss-net-data/193_test/imgs data/ch-test\(region_growth\)/images

In [4]:
# move ar data
!mkdir -p data/ar-train
!mkdir -p data/ar-test

!cp -r scss-net-data/171_train/171\(spoca\)/imgs/. data/ar-train/images 
!cp -r scss-net-data/171_train/171\(spoca\)/masks/. data/ar-train/masks

!cp -r scss-net-data/171_train/171\(zooniverse\)/imgs/. data/ar-train/images
!cp -r scss-net-data/171_train/171\(zooniverse\)/masks/. data/ar-train/masks

!cp -r scss-net-data/171_test/imgs/. data/ar-test/images 
!cp -r scss-net-data/171_test/masks\(spoca\)/. data/ar-test/masks

In [4]:
def create_contours(mask):
    
    width = mask.shape[1]
    height = mask.shape[0]
    # ret, thresh = cv2.threshold(mask, 127, 255, 0)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    coords = []
    for i, object in enumerate(contours):
        coords.append([])
        for point in object:
            new_x = (int(point[0][0])) / (width)
            new_y = (int(point[0][1])) / (height)
            coords[i].append((new_x, new_y))
    return coords

In [3]:
def convert_to_yolo(folder_path):
    masks_path = os.path.join(folder_path, 'masks')
    os.mkdir(os.path.join(folder_path, 'labels'))
    labels_path = os.path.join(folder_path, 'labels')

    png_files = [file for file in os.listdir(masks_path) if file.endswith('.png')]

    for png_file in png_files:
        input_path = os.path.join(masks_path, png_file)

        image_cv2 = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        coords_of_objects = create_contours(image_cv2)
    
        txt_file = os.path.splitext(png_file)[0] + '.txt'
        output_path = os.path.join(labels_path, txt_file)

        with open(output_path, 'w') as label_file:
            for i, object_coords in enumerate(coords_of_objects):
                coordinates_str = ' '.join(map(str, np.reshape(object_coords, -1)))
                string_to_write = f"0 {coordinates_str}"
                if i < (len(coords_of_objects) - 1):
                    string_to_write += "\n"
                label_file.write(string_to_write)

In [5]:
import cv2
from PIL import Image

def display_label(label_name):
    image = cv2.imread("./data/train(region_growth)/images/"+label_name+".png")

    with open("./data/train(region_growth)/labels/" + label_name + ".txt", 'r') as f:
        label = f.read()

    label_elements = label.strip().split(' ')

    coords = [(int(float(label_elements[i])*256), int(float(label_elements[i+1])*256)) for i in range(1, len(label_elements), 2)]
   
    for coord in coords:
        cv2.circle(image, coord, radius=0, color=(255, 0, 0), thickness=-1)
   
    cv2.imwrite("./test.png", image)


In [6]:
import cv2
import shutil

def convert_to_grayscale(folder_path):
    images_path = os.path.join(folder_path, 'images')
    grayscale_images_path = os.path.join(folder_path, 'images-gray')
    os.makedirs(grayscale_images_path)
    
    png_files = [file for file in os.listdir(images_path) if file.endswith('.png')]
    
    for img_path in png_files:
        image = cv2.imread(images_path + "/" + img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        cv2.imwrite(grayscale_images_path + "/" + img_path, gray_image)

    shutil.move(os.path.join(folder_path, 'images'), os.path.join(folder_path, 'images-col'))
    shutil.move(os.path.join(folder_path, 'images-gray'), os.path.join(folder_path, 'images'))
        

In [7]:
from pathlib import Path

def remove_invalid_labels_with_images(data_path):
    images_path = os.path.join(data_path, 'images')
    labels_path = os.path.join(data_path, 'labels')

    txt_files = [file for file in os.listdir(labels_path) if file.endswith('.txt')]
    
    for txt_file in txt_files:
        with open(labels_path +"/"+ txt_file, 'r') as f:
            label = f.read()
            png_file = os.path.splitext(txt_file)[0] + '.png'
            if(len(label.split()) < 5):
                print("removing... " + png_file)
                Path(images_path+"/" + png_file).unlink()
                print("removing... " + txt_file)
                Path(labels_path+"/" + txt_file).unlink()
        
            

In [10]:
convert_to_yolo("./data/ch-train(region_growth)")
convert_to_yolo("./data/ch-test(region_growth)")
convert_to_yolo("./data/ar-train")
convert_to_yolo("./data/ar-test")

In [11]:
remove_invalid_labels_with_images("./data/ch-train(region_growth)")
remove_invalid_labels_with_images("./data/ch-test(region_growth)")
remove_invalid_labels_with_images("./data/ar-train")
remove_invalid_labels_with_images("./data/ar-test")

removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193.png
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193.txt
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r180.png
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r180.txt
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r270.png
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r270.txt
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r90.png
removing... 2012_01_14__23_00_07_84__SDO_AIA_AIA_193_r90.txt
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193.png
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193.txt
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r180.png
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r180.txt
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r270.png
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r270.txt
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r90.png
removing... 2012_01_16__22_59_55_84__SDO_AIA_AIA_193_r90.txt
removing... 2012_01_18__23_00_19

In [8]:
import math
import os
import random
import shutil

def move_files(paths_list, dst_directory):
    for path in paths_list:
        src_path = path
        dst_path = os.path.join(dst_directory, os.path.basename(path))
        shutil.copyfile(src_path, dst_path)
        os.remove(src_path)


def generate_val_set(train_path, val_path, percent=0.1):
    images_path = os.path.join(train_path, "images")
    test_images_path = os.path.join(val_path, "images")
    test_labels_path = os.path.join(val_path, "labels")
    test_masks_path = os.path.join(val_path, "masks")
    os.makedirs(test_images_path, exist_ok=True)
    os.makedirs(test_labels_path, exist_ok=True)
    os.makedirs(test_masks_path, exist_ok=True)

    image_paths = []
    for root, _, files in os.walk(images_path):
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))

    random.shuffle(image_paths)

    test_num = math.floor(len(image_paths)*percent)
    test_images = image_paths[:test_num]
    test_labels = [path.replace("images","labels").replace("png", "txt") for path in test_images]
    test_masks = [path.replace("images","masks") for path in test_images]

    move_files(test_images, test_images_path)
    move_files(test_labels, test_labels_path)
    move_files(test_masks, test_masks_path)

In [13]:
generate_val_set("data/ch-train(region_growth)", "data/ch-val(region_growth)")
generate_val_set("data/ar-train", "data/ar-val")

In [17]:
convert_to_grayscale("./data/ch-test(region_growth)")
convert_to_grayscale("./data/ch-test(spoca)")
convert_to_grayscale("./data/ch-train(region_growth)")
convert_to_grayscale("./data/ch-train(zooniverse)")
convert_to_grayscale("./data/ch-val(region_growth)")

In [16]:
convert_to_grayscale("./data/ar-train")
convert_to_grayscale("./data/ar-test")
convert_to_grayscale("./data/ar-val")

In [18]:
# prepare SOHO dataset
# install mega-cmd https://mega.io/cmd#downloadapps
import zipfile

# os.chdir('/dev/bakalarka-yolo/') # needed if no memory on device

In [19]:
mega = Mega()
m = mega.login()
print("Downloading ...")
try:
    m.download_url('https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/o4Nk0D7I')
except PermissionError:
    pass
print("Unzipping ...")
with zipfile.ZipFile("171.zip","r") as zip_ref:
    zip_ref.extractall("data")
print("DONE")
# ch test imgs
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/o4Nk0D7I
# ch test masks
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/5lEAwRjA
# ch train imgs
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/MkcSiJwI 
# ch train masks
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/skMUnDyL 

/bin/bash: mega-get: command not found
/bin/bash: mega-get: command not found
/bin/bash: mega-get: command not found
/bin/bash: mega-get: command not found


In [15]:

# ar test imgs
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/kg1nzAxJ
# ar test masks
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/Z8111I6C
# ar train imgs
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/c19RwY4Q
# ar train masks
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/I18TSaTb

TRANSFERRING ||######################################||(695/695 MB: 100.00 %)  
Download finished: /dev/bakalarka-yolo/./AR_test_imgs.zip
TRANSFERRING ||######################################||(695/695 MB: 100.00 %)  
TRANSFERRING ||##########################################||(4/4 MB: 100.00 %)  
Download finished: /dev/bakalarka-yolo/./AR_test_masks.zip
TRANSFERRING ||##########################################||(4/4 MB: 100.00 %)  
TRANSFERRING ||####################################||(2784/2784 MB: 100.00 %)  
Download finished: /dev/bakalarka-yolo/./AR_train_imgs.zip
TRANSFERRING ||####################################||(2784/2784 MB: 100.00 %)  
TRANSFERRING ||########################################||(17/17 MB: 100.00 %)  
Download finished: /dev/bakalarka-yolo/./AR_train_masks.zip
TRANSFERRING ||########################################||(17/17 MB: 100.00 %)  


In [12]:
!mkdir -p data/ch-test\(SOHO\)
!mkdir -p data/ch-train\(SOHO\)
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/CH_test_imgs.zip","r") as zip_ref:
    zip_ref.extractall("data/ch-test(SOHO)")

!mv data/ch-test\(SOHO\)/test_imgs data/ch-test\(SOHO\)/images
    
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/CH_test_masks.zip","r") as zip_ref:
    zip_ref.extractall("data/ch-test(SOHO)")

!mv data/ch-test\(SOHO\)/test_masks data/ch-test\(SOHO\)/masks
    
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/CH_train_imgs.zip","r") as zip_ref:
    zip_ref.extractall("data/ch-train(SOHO)")

!mv data/ch-train\(SOHO\)/train_imgs data/ch-train\(SOHO\)/images

with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/CH_train_masks.zip","r") as zip_ref:
    zip_ref.extractall("data/ch-train(SOHO)")

!mv data/ch-train\(SOHO\)/train_masks data/ch-train\(SOHO\)/masks

convert_to_yolo("./data/ch-test(SOHO)")
remove_invalid_labels_with_images("./data/ch-test(SOHO)")
convert_to_grayscale("./data/ch-test(SOHO)")

convert_to_yolo("./data/ch-train(SOHO)")
remove_invalid_labels_with_images("./data/ch-train(SOHO)")
convert_to_grayscale("./data/ch-train(SOHO)")

generate_val_set("data/ch-train(SOHO)", "data/ch-val(SOHO)", 0.2)

In [14]:
!mkdir -p data/ar-test\(SOHO\)
!mkdir -p data/ar-train\(SOHO\)
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/AR_test_imgs.zip","r") as zip_ref:
    zip_ref.extractall("data/ar-test(SOHO)")

!mv data/ar-test\(SOHO\)/ARtest_imgs data/ar-test\(SOHO\)/images
    
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/AR_test_masks.zip","r") as zip_ref:
    zip_ref.extractall("data/ar-test(SOHO)")

!mv data/ar-test\(SOHO\)/ARtest_masks data/ar-test\(SOHO\)/masks
    
with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/AR_train_imgs.zip","r") as zip_ref:
    zip_ref.extractall("data/ar-train(SOHO)")

!mv data/ar-train\(SOHO\)/ARtrain_imgs data/ar-train\(SOHO\)/images

with zipfile.ZipFile("SOHO-segmentation-data/train_test_data/AR_train_masks.zip","r") as zip_ref:
    zip_ref.extractall("data/ar-train(SOHO)")

!mv data/ar-train\(SOHO\)/ARtrain_masks data/ar-train\(SOHO\)/masks

convert_to_yolo("./data/ar-test(SOHO)")
remove_invalid_labels_with_images("./data/ar-test(SOHO)")
convert_to_grayscale("./data/ar-test(SOHO)")

convert_to_yolo("./data/ar-train(SOHO)")
remove_invalid_labels_with_images("./data/ar-train(SOHO)")
convert_to_grayscale("./data/ar-train(SOHO)")

generate_val_set("data/ar-train(SOHO)", "data/ar-val(SOHO)", 0.2)